# Project Stage 3: Entity Matching

## Import relevant packages

In [ ]:
import os
import pandas as pd
import math

## Read the raw data from the source tables
Read the raw data which was extracted from IMDb and Rotten Tomatoes
Also read the Candidate Set extracted from Cloudmatcher

In [ ]:
table_A = 'IMDB_dataset'
table_B = 'RottenTomato_data'

dfa = pd.read_csv(table_A)
dfb = pd.read_csv(table_B)
dfc = pd.read_csv('Candidate_set')

## Data Cleaning
In this step we will perform data cleaning to enhance entity matching

### Remove irrelevant columns altogether

In [ ]:
del dfa['genre']
del dfa['Up_System']
del dfa['Release Date']
del dfa['Rating']
del dfa['Runtime']

del dfb['genre']
del dfb['Up_System']
del dfb['Release Date']
del dfb['Rating']
del dfb['Runtime']

### Convert object datatype to String for consistency

In [ ]:
dfa.ReleaseYear = dfa.ReleaseYear.astype(str)
dfb.ReleaseYear = dfb.ReleaseYear.astype(str)

## Candidate set to Full Entity conversion
In this step we read the id references from candidate sets and corresponding entities from the source tables

In [ ]:
table_ac = []
table_bc = []
for items in dfc.values.tolist():
    table_ac.append(dfa[dfa['_id'] == items[0]].values.tolist()[0])
    table_bc.append(dfb[dfb['_id'] == items[1]].values.tolist()[0])
    
# Convert list of tuples to dataframe and set column names and indexes
table_ac = pd.DataFrame(table_ac, columns = ['_id','Name','ReleaseYear','Runtime','Director Name','Certificate']) 
table_bc = pd.DataFrame(table_bc, columns = ['_id','Name','ReleaseYear','Runtime','Director Name','Certificate']) 

## Blocking Rule 1
In this rule, we check if the release years of the entities has an absolute difference of less than equal to 1. If the difference is greater than 1, we BLOCK the match.

In [ ]:
c_prime_dict = []

for ind in range(table_ac.shape[0]):
    a_item = table_ac.iloc[[ind]]
    b_item = table_bc.iloc[[ind]]
    a_rel_year_str = a_item['ReleaseYear'].item()
    b_rel_year_str = b_item['ReleaseYear'].item()
        
    # Adding try catch here as the data column can have incorrect values
    # like nan, NAN, HbO, Random, etc.
    try:
        a_rel_year_int = int(a_rel_year_str)
    except ValueError:
        a_rel_year_int = 0
        
    try:
        b_rel_year_int = int(b_rel_year_str)
    except ValueError:
        b_rel_year_int = 0
    
    if abs(a_rel_year_int - b_rel_year_int) <= 1:
        a = a_item['_id']
        b = b_item['_id']
        row_data = []
        row_data.append(a.item())
        row_data.append(b.item())
        c_prime_dict.append(row_data)
        
c_prime = pd.DataFrame(c_prime_dict, columns = ['A_id','B_id']) 
c_prime.to_csv('candidate_set_after_first_block_rule_on_rel_year.csv', index=False)

### Defining a Jaccard measure function for second blocking rule

In [ ]:
def check_jaccard(string1,string2):
    setA=[]
    setB=[]
    count=0
    for i in range(0, len(string1) - 2):
        setA.append(string1[i : i + 3])
    for i in range(0, len(string2) - 2):
        setB.append(string2[i : i + 3])
    for item in setA:
        if item in setB:
            count = count + 1
    return float(count) / (len(string1) + len(string2) - count)

## Blocking Rule 2
We apply blocking rule 2 on the output of blocking rule 1. In this rule, we check the following
1. Jaccard measure of the 3 grams on the Movie Name >= 0.6
2. Jaccard measure of the 3 grams on the Movie Name is between (0.3, 0.6)
- Release year should have an absolute difference of 1 year
- Jaccard measure between Director Name > 0.6

The second part of this rule is applied to retrieve the following kind of entity matches:
1. MISSION: IMPOSSIBLE II v/s MISSION: IMPOSSIBLE 2
2. LES QUATRE CENTS COUPS v/s THE 400 BLOWS (LES QUATRE CENTS COUPS)

In [ ]:
c_prime_dict_after_name_blocking = []

for i in range(0,len(c_prime)):
    a_loc = c_prime.iloc[i,0]
    b_loc = c_prime.iloc[i,1]
    stringA=str(dfa.iloc[a_loc,1]).upper()
    stringB=str(dfb.iloc[b_loc,1]).upper()
    jacc_score = check_jaccard('##' + stringA + '##','##' + stringB + '##')
    
    if jacc_score >= 0.6:
        a_item = dfa.iloc[[a_loc]]
        b_item = dfb.iloc[[b_loc]]
        a = a_item['_id']
        b = b_item['_id']
        row_data = []
        row_data.append(a.item())
        row_data.append(b.item())
        c_prime_dict_after_name_blocking.append(row_data)

    elif jacc_score < 0.6 and jacc_score > 0.3:
        a_item = dfa.iloc[[a_loc]]
        b_item = dfb.iloc[[b_loc]]
        a_rel_year_str = a_item['ReleaseYear'].item()
        b_rel_year_str = b_item['ReleaseYear'].item()

        try:
            a_rel_year_int = int(a_rel_year_str)
        except ValueError:
            a_rel_year_int = 0

        try:
            b_rel_year_int = int(b_rel_year_str)
        except ValueError:
            b_rel_year_int = 0

        if abs(a_rel_year_int - b_rel_year_int) <= 1:
            
            dir_A = a_item['Director Name'].item().upper()
            dir_B = b_item['Director Name'].item().upper()
            jacc_score_dir = check_jaccard('##' + dir_A + '##','##' + dir_B + '##')
            if jacc_score_dir > 0.6:
                
                a = a_item['_id']
                b = b_item['_id']
                row_data = []
                row_data.append(a.item())
                row_data.append(b.item())
                c_prime_dict_after_name_blocking.append(row_data)
                
df_c_prime_dict_after_name_blocking = pd.DataFrame(c_prime_dict_after_name_blocking, columns = ['A_id','B_id'])
df_c_prime_dict_after_name_blocking.to_csv('candidate_set_after_second_block_rule.csv', index=False)

In [ ]:
df_c_prime_dict_after_name_blocking.drop_duplicates(inplace=True)

## Perform debug_blocker step
Here we performed the debug_blocker step and checked that out of the 200 entries reported, only 2 were True Positives. Thus our blocking rules are not dropping lot of TP.

## Perform Manual Labeling on 50 samples
Here we pick 50 samples from our final candidate set and manually label them. We got 2 FP out of 50, thus the density is 48/50 = 0.96

In [ ]:
sampled_50 = df_c_prime_dict_after_name_blocking.sample(n=50, random_state=10)

## Perform Manual Labeling on 400 samples
Here we pick 400 samples from our final candidate set and manually label them. We got 5 FP out of 400, thus the density is 395/400 = 0.9875

By passing the same seed value, our first 50 samples remained the same.

In [ ]:
sampled_400 = df_c_prime_dict_after_name_blocking.sample(n=400, random_state=10)

## Saving final Candidate Set

In [ ]:
sampled_400.to_csv('final_candidate_set.csv',index=False)